# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [23]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [24]:
city_data_df = pd.read_csv("weather_data.csv")
city_data_df.head()


,Unnamed: 0,Date,City,Country,Longitude,Latitude,Cloudiness,Max Temp,Humidity,Wind Speed
0,0,1616804300,Karaton,ID,106.1045,-6.3424,20,82.99,83,3.44
1,1,1616804300,Tiksi,RU,128.8694,71.6872,100,10.83,91,6.51
2,2,1616804301,Harwich,GB,1.2844,51.9419,56,42.01,71,10.00
3,3,1616804302,Samarai,PG,150.6667,-10.6167,93,83.80,75,11.34
4,4,1616804302,Busselton,AU,115.3333,-33.6500,69,57.00,87,5.50


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
#config googlemaps

gmaps.configure(api_key = g_key)

In [28]:
#Humidity Heatmap

locations = city_data_df[["Latitude", "Longitude"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heatmap = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 300, point_radius = 5)

fig.add_layer(heatmap)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
relevant_cities = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                   & (city_data_df["Wind Speed"]< 10) \
                                   & (city_data_df["Cloudiness"]== 0)].dropna()
relevant_cities

,Unnamed: 0,Date,City,Country,Longitude,Latitude,Cloudiness,Max Temp,Humidity,Wind Speed
77,77,1616804332,Arraial do Cabo,BR,-42.0278,-22.9661,0,78.80,83,6.91
154,154,1616804368,Sonāmukhi,IN,87.4167,23.3000,0,74.23,22,4.27
156,156,1616804369,Caravelas,BR,-39.2481,-17.7125,0,75.67,78,5.06
164,164,1616804371,Belmonte,BR,-38.8828,-15.8631,0,78.80,73,8.05
173,173,1616804374,Salalah,OM,54.0924,17.0151,0,73.40,27,5.75
175,175,1616804375,Vila Velha,BR,-40.2925,-20.3297,0,77.00,83,3.44
193,193,1616804383,Saint-Paul,RE,55.2707,-21.0096,0,75.00,88,4.61
311,311,1616804439,Saquarema,BR,-42.5103,-22.9200,0,78.80,83,6.91
341,341,1616804402,Makkah al Mukarramah,SA,39.8261,21.4267,0,71.20,49,6.78
392,392,1616804473,Doha,QA,51.5333,25.2867,0,78.01,78,2.30


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
hotel_df = relevant_cities[['City', 'Country', 'Longitude', 'Latitude']].copy()
hotel_df['Hotel Name'] = ''
hotel_df

,City,Country,Longitude,Latitude,Hotel Name
77,Arraial do Cabo,BR,-42.0278,-22.9661,
154,Sonāmukhi,IN,87.4167,23.3000,
156,Caravelas,BR,-39.2481,-17.7125,
164,Belmonte,BR,-38.8828,-15.8631,
173,Salalah,OM,54.0924,17.0151,
175,Vila Velha,BR,-40.2925,-20.3297,
193,Saint-Paul,RE,55.2707,-21.0096,
311,Saquarema,BR,-42.5103,-22.9200,
341,Makkah al Mukarramah,SA,39.8261,21.4267,
392,Doha,QA,51.5333,25.2867,


In [33]:
parameters = {'radius' : 5000, 'types': 'lodging', 'key': g_key}

#iterate through loop

for index, row in hotel_df.iterrows():
    
    lat = row["Latitude"]
    long = row["Longitude"]
    
    parameters["location"] = f'{lat},{long}'
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #requests api and 
    
    hotels = requests.get(base_url, params = parameters)
    
    hotels = hotels.json()
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels["results"][0]["name"]
    except (KeyError, IndexError):
        print(f'No Result.../ Skipping.')
    
hotel_df

No Result.../ Skipping.
No Result.../ Skipping.
No Result.../ Skipping.
No Result.../ Skipping.
No Result.../ Skipping.
No Result.../ Skipping.
No Result.../ Skipping.
No Result.../ Skipping.
No Result.../ Skipping.
No Result.../ Skipping.
No Result.../ Skipping.
No Result.../ Skipping.
No Result.../ Skipping.
No Result.../ Skipping.
No Result.../ Skipping.


,City,Country,Longitude,Latitude,Hotel Name
77,Arraial do Cabo,BR,-42.0278,-22.9661,
154,Sonāmukhi,IN,87.4167,23.3000,
156,Caravelas,BR,-39.2481,-17.7125,
164,Belmonte,BR,-38.8828,-15.8631,
173,Salalah,OM,54.0924,17.0151,
175,Vila Velha,BR,-40.2925,-20.3297,
193,Saint-Paul,RE,55.2707,-21.0096,
311,Saquarema,BR,-42.5103,-22.9200,
341,Makkah al Mukarramah,SA,39.8261,21.4267,
392,Doha,QA,51.5333,25.2867,


In [35]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [36]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))